# IBM Ponder This - September 2024 

## Problem Statement

We will call two triangles "sibling pairs" if they are two non-identical triangles with integer side lengths respectively $(a,b,c_1)$ and $(a,b,c_2)$, where $c_1$ and $c_2$ are the only side of each triangle with a different length from the other triangle, since there are two common side lengths $a$,$b$ **and** both triangles must have a common nonzero area.

An example is the pair of triangles with side lengths $(5,5,6)$ and $(5,5,8)$ . The area of both is 12. Another example is $(7,4,7)$ and $(7,4,9)$ with area $\sqrt{180}$.

For some $a$ and $b$ , there may not be any sibling pair at all (e.g., $a=3$ and $b=2$) and for some, there may be more than one. For example, if $a=24$ and $b=23$, three sibling pairs exist with the following side lengths:

- $(24,23,19)$ and $(24,23,43)$, area = $\sqrt{41580}$
- $(24,23,23)$ and $(24,23,41)$, area = $\sqrt{55440}$
- $(24,23,29)$ and $(24,23,37)$, area = $\sqrt{71820}$

Your goal: Find two integers $a$, $b$ such that $a>b$ and there are **exactly** 50 sibling pairs with common side lengths $a$ and $b$.

**A bonus** "*" will be given for solving the above problem with the additional constraint that at least two of those 50 sibling pairs must have **integer** areas. The areas of the remaining pairs may be irrational.

## Solution

For some sides $a$ and $b$ with $a > b$, there are always two triangles with the same area (one obtuse and one acute) and this area, $A$, is given by

\begin{equation}
    A = \frac{1}{2}ab\sin({\theta}) = \frac{1}{2}ab\sin({180 - \theta})
\end{equation}

where $\theta$ is the angle in degrees between $a$ and $b$ in the acute triangle (and $180 - \theta$ is the angle in the corresponding obtuse triangle). 

Let $c_1$ and $c_2$ represent the sides of the two triangles with same area. From the cosine law, we have

\begin{align}
    c_1 &= \sqrt{a^2 + b^2 - ab\cos(\theta)} \\
    c_2 &= \sqrt{a^2 + b^2 + ab\cos(\theta)}
\end{align}

where the second equation comes from the fact that $\cos(\theta) = -\cos(180 - \theta)$.

Therefore, $c_1$ is an integer only if $a^2 + b^2 - ab\cos(\theta)$ is a perfect square and $c_2$ is an integer only if $a^2 + b^2 + ab\cos(\theta)$ is a perfect square. 

We can use the above properties to restrict the range of $c$ values that we consider for a given $(a, b)$. We first precompute all the squares below $a_{\text{max}}^2 + b_{\text{max}}^2$ where $a_{\text{max}}$ and $b_{\text{max}}$ are the maximum values we will consider for $a$ and $b$. We do not need squares above $a_{\text{max}}^2 + b_{\text{max}}^2$ because this corresponds to a right triangle and we only need squares to generate acute triangles.

For a given $(a, b)$, we loop through the squares in the range from $(|a - b| + 1)^2$ to $a^2 + b^2$. To avoid unnecessary computations, we identify the index of the first relevant square with a binary search. We set $c_1^2$ equal to the square we currently process and compute the corresponding $c_2^2$ value as

\begin{equation}
    c_2^2 = a^2 + b^2 + 2ab\cos(\theta)
\end{equation}
    
where

\begin{equation}
    \cos(\theta) = \frac{a^2 + b^2 - c_1^2}{2ab}.
\end{equation}

If $c_2$ is an integer and $(a, b, c)$ form a valid triangle (i.e., no side is longer than the sum of the two other sides), we have found a sibling pair. 

Here we generate $a$ randomly in the range 1 million to 50 million and generate $b$ as $ua$ where $u \sim \text{Unif}(\frac{1}{2}, 1)$. As python can be quite slow, we use numba to speed up the computations.

With the tricks above, we manage to get a solution pretty quickly even when generating $a$ and $b$ randomly.

For the bonus, for each $(a, b)$ combination with exactly 50 sibling pairs, I tried computing the area of each triangle with high precision using Heron's formula, 

\begin{equation}
    A = \sqrt{s(s - a)(s - b)(s - c)}
\end{equation}

where $s$ is the semiperimeter given by

\begin{equation}
    s = \frac{a + b + c}{2}.
\end{equation}

Unfortunately, I could not find any integer area. I guess it would be necessary to use properties of Heronian triangles.


In [2]:
import numpy as np
from numba import njit
from decimal import Decimal, getcontext

getcontext().prec = 50

def precompute_squares(max_val):
    """ Returns a list of all squares up to max_val. """
    squares = []
    i = 0
    while i**2 <= max_val:
        squares.append(i**2)
        i += 1
    return squares

@njit
def binary_search_squares(squares, target):
    """ Perform binary search to find the index of the first square >= target. """
    low, high = 0, len(squares) - 1
    while low <= high:
        mid = (low + high) // 2
        if squares[mid] < target:
            low = mid + 1
        elif squares[mid] > target:
            high = mid - 1
        else:
            return mid  # Found the exact square
    return low  # Return the index where target should be inserted (first larger element)

@njit
def find_valid_c_pairs(a, b, squares):
    """ Count the number of sibling pairs """
    valid_pairs = 0
    # Relevant square range
    min_c_squared = (abs(a - b) + 1)**2
    max_c_squared = a**2 + b**2
    # Get index of first relevant square
    start_index = binary_search_squares(squares, min_c_squared)

    for c1_squared in squares[start_index:]:
        # Break if side makes triangle obtuse
        if c1_squared > max_c_squared:
            break
        # Compute c_2^2 using cosine law
        cos_theta = (a**2 + b**2 - c1_squared) / (2 * a * b)
        c2_squared = a**2 + b**2 + 2 * a * b * cos_theta
        # Check if c_2^2 is a perfect square
        if c2_squared == int(c2_squared) and c2_squared != c1_squared:
            # Check if c_2 is valid
            c2 = np.sqrt(c2_squared)
            if c2 == int(c2) and is_valid_triangle(a, b, c2):
                valid_pairs += 1
    return valid_pairs

@njit
def get_valid_c(a, b, squares):
    """ Count the number of sibling pairs """
    res = np.zeros(50)
    valid_pairs = 0
    # Relevant square range
    min_c_squared = (abs(a - b) + 1)**2
    max_c_squared = a**2 + b**2
    # Get index of first relevant square
    start_index = binary_search_squares(squares, min_c_squared)

    for c1_squared in squares[start_index:]:
        # Break if side makes triangle obtuse
        if c1_squared > max_c_squared:
            break
        # Compute c_2^2 using cosine law
        cos_theta = (a**2 + b**2 - c1_squared) / (2 * a * b)
        c2_squared = a**2 + b**2 + 2 * a * b * cos_theta
        # Check if c_2^2 is a perfect square
        if c2_squared == int(c2_squared) and c2_squared != c1_squared:
            # Check if c_2 is valid
            c2 = np.sqrt(c2_squared)
            if c2 == int(c2) and is_valid_triangle(a, b, c2):
                res[valid_pairs] = c2
                valid_pairs += 1
    return res

@njit
def counter_integer_area(a, b, squares):
    """ Count the number of sibling pairs with integer area"""
    n_integer_area = 0
    # Relevant square range
    min_c_squared = (abs(a - b) + 1)**2
    max_c_squared = a**2 + b**2
    # Get index of first relevant square
    start_index = binary_search_squares(squares, min_c_squared)
    for c1_squared in squares[start_index:]:
        # Break if side makes triangle obtuse
        if c1_squared > max_c_squared:
            break
        # Compute c_2^2 using cosine law
        cos_theta = (a**2 + b**2 - c1_squared) / (2 * a * b)
        c2_squared = a**2 + b**2 + 2 * a * b * cos_theta
        # Check if c_2^2 is a perfect square
        if c2_squared == int(c2_squared) and c2_squared != c1_squared:
            c2 = np.sqrt(c2_squared)
            # Check if c_2 is valid
            if c2 == int(c2) and is_valid_triangle(a, b, c2):
                # Compute area
                area = heron_formula(a, b, c2)
                if abs(area - int(area)) < 10**(-12):
                    n_integer_area += 1
    return n_integer_area

def heron_area_high_precision(a, b, c):
    # Convert inputs to Decimal for high precision calculations
    a, b, c = Decimal(a), Decimal(b), Decimal(c)
    s = (a + b + c) / 2
    # Calculate the area using Heron's formula with high precision
    area = (s * (s - a) * (s - b) * (s - c)).sqrt()
    return area

@njit
def heron_formula(a, b, c):
    """ Compute area of triangle given 3 side lengths """
    s = (a + b + c) / 2
    area = np.sqrt(s * (s - a) * (s - b) * (s - c))
    return area

@njit
def is_valid_triangle(a, b, c):
    """ Check if a triangle can be formed with 3 side lengths """
    return a + b > c and a + c > b and b + c > a


max_a_b = 100000000
squares = precompute_squares(max_a_b**2)
squares = np.array(squares)

for _ in range(100000000):
    a = np.random.randint(1000000, 50000000)
    b = int(np.random.uniform(0.5, 1) * a)
    valid_pairs = find_valid_c_pairs(a, b, squares)
    if valid_pairs == 50:
        # cs = get_valid_c(a, b, squares)
        # n_zero_area = 0
        # for c in cs:
        #     area = heron_area_high_precision(a, b, c)
        #     if abs(area - int(area)) < 10**(-10):
        #         n_zero_area += 1
        # print(a, b, c, n_zero_area)
        print(a, b)
        break

36644223 23626786
